In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html, ctx      #added callback context function
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#  CRUD Python module file name and class name
from CRUD_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

#connection varaibles to access the database
username = "aacuser"
password = "DogsAreCool"
host = 'nv-desktop-services.apporto.com'
port = 31018
db = 'ACC'
col = 'animals'

#instance of AnimalShelter class
db = AnimalShelter(username, password,host, port, db, col)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='buttonRow',
            style={'display': 'flex'},
                children=[
                    html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
                    html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
                ]),
    dash_table.DataTable(id='datatable-id',
                         columns=[{'name': i, 'id': i, 'deletable': False, 'selectable': True}
                                  for i in df.columns],
                         data=df.to_dict('records'),
                         editable=False,
                         filter_action='native',
                         sort_action='native',
                         sort_mode='multi',
                         column_selectable=False,
                         row_selectable='single',
                         row_deletable=False,
                         selected_columns=[],
                         selected_rows=[0],
                         page_action='native',
                         page_current=0,
                         page_size=10
                        ),
     html.Br(),
     html.Hr(),
     html.Div(id='map-id',
            className='col s12 m6'),
     html.Div('E.Jennings-Dash Application')  #unique identifier
])

#############################################
# Interaction Between Components / Controller
#############################################
#callback to use filter buttons
@app.callback(Output('datatable-id', 'data'),
             [Input('submit-button-one', 'n_clicks'),
              Input('submit-button-two', 'n_clicks')
             ])
def on_click(button1, button2): 
    """
    Method to activate buttons when clicked
    :param data: buttons 1 and 2
    :return: filtered data
    """
    # start case
    df = pd.DataFrame.from_records(db.read({}))
    
    # uses the dash callback context function to determine which callback input was changed
    if ('submit-button-one' == ctx.triggered_id):
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Cat'}))
    elif ('submit-button-two' == ctx.triggered_id):
        df = pd.DataFrame.from_records(db.read({'animal_type' : 'Dog'}))
    
    # Cleanup Mongo _id field
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    """
    Method to update geolocation map
    :param data: the index of the row and the data's location
    :return: map of the data's location
    """
    dff = pd.DataFrame.from_dict(viewData)
    if index is None:
        row = 0
    else:
        row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
            center = [30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id='base-layer-id'),
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1('Animal Name'),
                    html.P(dff.iloc[row,9]),
                    html.Div(id='map-id',
                             className='col s12 m6')
                ])
            ])
        ])
    ]
    

app.run_server(debug=True)

Dash app running on http://127.0.0.1:14520/
